# Technische Universität Berlin #
**Fakultät II - Mathematik und Naturwissenschaften**

**Seminar Stochastische Partielle Differentialgleichungen**

*Prof. Dr. Wilhelm Stannat*

*Yang Bai*

*Studiengang: Wissenschaftliches Rechnen (Master)*

*SoSe 2021*

## Import all necessary modules

In [1]:
import numpy as np
from scipy.interpolate import lagrange
from prettytable import PrettyTable

# Revsion: The $\theta$ method for FBSDEs model:
Let us use explicit euler method to calculate the numerical results

**(1) Consider the decoupled FBSDE:**
$$
\left\{\begin{array}{l}
\mathrm{d} X_{t}=\frac{1}{1+2 \exp \left(t+X_{t}\right)} \mathrm{d} t+\frac{\exp \left(t+X_{t}\right)}{1+\exp \left(t+X_{t}\right)} \mathrm{d} W_{t} \\
-\mathrm{d} Y_{t}=\left(-\frac{2 Y_{t}}{1+2 \exp \left(t+X_{t}\right)}-\frac{1}{2}\left(\frac{Y_{t} Z_{t}}{1+\exp \left(t+X_{t}\right)}-Y_{t}^{2} Z_{t}\right)\right) \mathrm{d} t-Z_{t} \mathrm{~d} W_{t}
\end{array}\right.
$$

We assume $
x_{0}=0, T=1
$,
with the terminal conditions:$$
Y_{T}=\frac{\exp \left(T+X_{T}\right)}{1+\exp \left(T+X_{T}\right)}
$$
The analytical solution: $$
Y_{t}=\frac{\exp \left(t+X_{t}\right)}{1+\exp \left(t+X_{t}\right)}, \quad Z_{t}=\frac{\left(\exp \left(t+X_{t}\right)\right)^{2}}{\left(1+\exp \left(t+X_{t}\right)\right)^{3}}
$$
then we have the below definition:

In [2]:
def g(x,T):
    return np.exp(T+x)/(1+np.exp(T+x))
def Z_exact(x,t):
    return (np.exp(t+x))**2/(1+np.exp(t+x))**3
def f(y,x,t,z):
    return (-2*y/(1+2*np.exp(t+x))-0.5*(y*z/(1+np.exp(t+x))-y*y*z))
def sigma(x,t):
    return np.exp(t+x)/(1+np.exp(t+x))
def beta(x,t):
    return 1/(1+2*np.exp(t+x))

In [3]:
def expectation(I,x,sigma,beta,t,delta_t):
    a,w=np.polynomial.hermite.hermgauss(10)
    x = np.expand_dims(x, axis=1)
    a = np.expand_dims(a, axis=0)
    E_1=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a)),w)/np.sqrt(np.pi)
    E_2=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a))*np.sqrt(2*delta_t)*a,w)/np.sqrt(np.pi)
    return E_1,E_2

In [4]:
def expectation_fm(I,x,sigma,beta,delta_t,NE,f,z,t):
    e3=0
    for i in range(0,NE):
        e3+=f(I(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)),x,t,z(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)))
    e3/=NE
    return e3

In [5]:
x=np.arange(0.9,1.1,0.05)

In [6]:
x

array([0.9 , 0.95, 1.  , 1.05, 1.1 ])

In [7]:
N_=[1,2,3,4,5,6,7,8,9];T0=0;T=1;
y=np.zeros(len(N_))
z=np.zeros(len(N_))
for N in range(len(N_)):
    t,delta_t=np.linspace(T0,T,num=N_[N]+1,endpoint=True,retstep=True)
    Y=np.zeros((N_[N]+1,len(x)))
    Z=np.zeros((N_[N]+1,len(x)))
    Y[-1]=g(x,T)
    Z[-1]=Z_exact(x,T)
    I=lagrange(x,g(x,T))
    IZ=lagrange(x,Z_exact(x,T))
    for n in range(N_[N]-1,-1,-1): 
        E_1,E_2=expectation(I,x,sigma,beta,t[n],delta_t)
        E_3 = expectation_fm(I,x,sigma,beta,delta_t,500,f,IZ,t[n])
        Z[n]=E_2/delta_t
        Y[n]=E_1+delta_t*E_3
        I=lagrange(x,Y[n])
        IZ=lagrange(x,Z[n])
    y[N]=Y[0][2]
    z[N]=Z[0][2]

In [8]:
exact_y,exact_z=g(1,0),Z_exact(1,0)

In [9]:
table = PrettyTable()
table.field_names = ["N","y_exact","y","z_exact","z","|y-y_exact|", "|z-z_exact|"]
for i in range(len(N_)):
    table.add_row([N_[i],"{:.8f}".format(exact_y),y[i],"{:.8f}".format(exact_z),z[i],"{:.8f}".format(abs(y[i]-exact_y)),"{:.8f}".format(abs(z[i]-exact_z))])
print(table)

+---+------------+--------------------+------------+---------------------+-------------+-------------+
| N |  y_exact   |         y          |  z_exact   |          z          | |y-y_exact| | |z-z_exact| |
+---+------------+--------------------+------------+---------------------+-------------+-------------+
| 1 | 0.73105858 | 0.6263826098191397 | 0.14373484 | 0.07623770088075846 |  0.10467597 |  0.06749714 |
| 2 | 0.73105858 | 0.6830700114265429 | 0.14373484 | 0.11963460016088456 |  0.04798857 |  0.02410024 |
| 3 | 0.73105858 | 0.7008718292475297 | 0.14373484 | 0.12994032785445966 |  0.03018675 |  0.01379451 |
| 4 | 0.73105858 | 0.7091652967557999 | 0.14373484 | 0.13501950900884863 |  0.02189328 |  0.00871533 |
| 5 | 0.73105858 | 0.7141844870931404 | 0.14373484 | 0.13798693327921352 |  0.01687409 |  0.00574791 |
| 6 | 0.73105858 | 0.717473673915016  | 0.14373484 | 0.14029546656880001 |  0.01358490 |  0.00343937 |
| 7 | 0.73105858 | 0.719992105417779  | 0.14373484 | 0.14164028261515768 

**Reference:** 

(1)Tao Tang, Weidong ZhaoandTao Zhou, Deferred Correction Methods for Forward Backward Stochastic Differential Equations, Numerical Mathematics: Theory, Methods and Applications , Volume 10 , Issue 2 , May 2017 , pp. 222 - 242

DOI: https://doi.org/10.4208/nmtma.2017.s02

(2)Weidong Zhao, Yu Fu, and Tao Zhou, New Kinds of High-Order Multistep Schemes for Coupled Forward Backward Stochastic Differential Equations, SIAM J. Sci. Comput., 36(4), A1731–A1751. (21 pages)

Read More: https://epubs.siam.org/doi/abs/10.1137/130941274

**(2) Consider the decoupled FBSDE:**$$
\begin{cases}dx_{t}=x_{t}dt+0.1x_{t}dW_{t}&\\ -dy_{t}=\left( 10z_{t}+\frac{y_{t}}{200} \right)  dt-z_{t}dW_{t}&\end{cases} 
$$

We assume $
x_{0}=1, T=1
$,
with the terminal conditions:$$
y_{T}=e^{-x}
$$
The analytical solution: $$
\left( y_{t},z_{t}\right)  =\left( e^{-x},-\frac{xe^{-x}}{10} \right)  
$$
then we have the below definition:

In [10]:
def g(x,t):
    return np.exp(-x)

In [11]:
def f(y,x,t,z):
    return -((z/0.1)+0.5*0.1*0.1*y)

In [12]:
def Z_exact(x,t):
    return -np.exp(-x)*0.1*x

In [13]:
def sigma(x,t):
    return 0.1*x

In [14]:
def beta(x,t):
    return x

In [15]:
def expectation(I,x,sigma,beta,t,delta_t):
    a,w=np.polynomial.hermite.hermgauss(10)
    x = np.expand_dims(x, axis=1)
    a = np.expand_dims(a, axis=0)
    E_1=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a)),w)/np.sqrt(np.pi)
    E_2=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a))*np.sqrt(2*delta_t)*a,w)/np.sqrt(np.pi)
    return E_1,E_2

In [16]:
def expectation_fm(I,x,sigma,beta,delta_t,NE,f,z):
    e3=0
    for i in range(0,NE):
        e3+=f(I(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)),x,t,z(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)))
    e3/=NE
    return e3

In [17]:
x=np.arange(0.9,1.1,0.05)

In [18]:
x

array([0.9 , 0.95, 1.  , 1.05, 1.1 ])

In [19]:
N_=[32,64,128,256,512];T0=0;T=1;
y=np.zeros(len(N_))
z=np.zeros(len(N_))
for N in range(len(N_)):
    t,delta_t=np.linspace(T0,T,num=N_[N]+1,endpoint=True,retstep=True)
    Y=np.zeros((N_[N]+1,len(x)))
    Z=np.zeros((N_[N]+1,len(x)))
    Y[-1]=g(x,T)
    Z[-1]=Z_exact(x,T)
    I=lagrange(x,g(x,T))
    IZ=lagrange(x,Z_exact(x,T))
    for n in range(N_[N]-1,-1,-1): 
        E_1,E_2=expectation(I,x,sigma,beta,t[n],delta_t)
        E_3 = expectation_fm(I,x,sigma,beta,delta_t,500,f,IZ)
        Z[n]=E_2/delta_t
        Y[n]=E_1+delta_t*E_3
        I=lagrange(x,Y[n])
        IZ=lagrange(x,Z[n])
    y[N]=Y[0][2]
    z[N]=Z[0][2]

In [20]:
exact_y,exact_z=g(1,0),Z_exact(1,0)

In [21]:
table = PrettyTable()
table.field_names = ["N","y_exact","y","z_exact","z","|y-y_exact|", "|z-z_exact|"]
for i in range(len(N_)):
    table.add_row([N_[i],"{:.8f}".format(exact_y),y[i],"{:.8f}".format(exact_z),z[i],"{:.8f}".format(abs(y[i]-exact_y)),"{:.8f}".format(abs(z[i]-exact_z))])
print(table)

+-----+------------+---------------------+-------------+-----------------------+-------------+-------------+
|  N  |  y_exact   |          y          |   z_exact   |           z           | |y-y_exact| | |z-z_exact| |
+-----+------------+---------------------+-------------+-----------------------+-------------+-------------+
|  32 | 0.36787944 | 0.36176979788518737 | -0.03678794 | -0.036952601504824434 |  0.00610964 |  0.00016466 |
|  64 | 0.36787944 | 0.36496121331788506 | -0.03678794 |  -0.03670272358811309 |  0.00291823 |  0.00008522 |
| 128 | 0.36787944 | 0.36644705135081207 | -0.03678794 |  -0.03656558249022643 |  0.00143239 |  0.00022236 |
| 256 | 0.36787944 |  0.3671661116619773 | -0.03678794 |  -0.03649495968640918 |  0.00071333 |  0.00029298 |
| 512 | 0.36787944 | 0.36751995227090645 | -0.03678794 |  -0.03645818430962587 |  0.00035949 |  0.00032976 |
+-----+------------+---------------------+-------------+-----------------------+-------------+-------------+


**Reference:** Kong Tao, Weidong Zhao , Shige Peng, High-Accuracy Numerical Schemes for 2nd-order Forward-Backward Stochastic Differential Equations and its Application, Doctor of Philosophy Thesis at ShanDong University, 2015

**(3) Consider the pure BSDE:**
    $$
\left\{\begin{array}{l}
d y_{t}=\left(\frac{y_{t}}{2}-z_{t}\right) d t-z_{t} d W_{t}, \quad 0 \leq t<T \\
y_{T}=\sin \left(W_{T}+T\right)
\end{array}\right.
$$

We assume $
x_{0}=0, T=1,
$with the analytical solution: $$\left(y_{t}, z_{t}\right)=\left(\sin \left(W_{t}+t\right), \cos \left(W_{t}+t\right)\right)$$
then we have the below definition:

In [22]:
def g(x,t):
    return np.sin(x+t)

In [23]:
def beta(x,t):
    return 0

In [24]:
def Z_exact(x,t):
    return np.cos(x+t)

In [25]:
def f(y,x,t,z):
    return y/2-z

In [26]:
def sigma(x,t):
    return 1

In [27]:
def expectation(I,x,sigma,beta,t,delta_t):
    a,w=np.polynomial.hermite.hermgauss(10)
    x = np.expand_dims(x, axis=1)
    a = np.expand_dims(a, axis=0)
    E_1=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a)),w)/np.sqrt(np.pi)
    E_2=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a))*np.sqrt(2*delta_t)*a,w)/np.sqrt(np.pi)
    return E_1,E_2

In [28]:
def expectation_fm(I,x,sigma,beta,delta_t,NE,f,z):
    e3=0
    for i in range(0,NE):
        e3+=f(I(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)),x,t,z(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)))
    e3/=NE
    return e3

In [29]:
x=np.arange(-0.1,0.1,0.05)
x

array([-0.1 , -0.05,  0.  ,  0.05])

In [30]:
g(x,0)

array([-0.09983342, -0.04997917,  0.        ,  0.04997917])

In [31]:
N_=[1,2,3,4,5];T0=0;T=1;
y=np.zeros(len(N_))
z=np.zeros(len(N_))
for N in range(len(N_)):
    t,delta_t=np.linspace(T0,T,num=N_[N]+1,endpoint=True,retstep=True)
    Y=np.zeros((N_[N]+1,len(x)))
    Z=np.zeros((N_[N]+1,len(x)))
    Y[-1]=g(x,T)
    Z[-1]=Z_exact(x,T)
    I=lagrange(x,g(x,T))
    IZ=lagrange(x,Z_exact(x,T))
    for n in range(N_[N]-1,-1,-1): 
        E_1,E_2=expectation(I,x,sigma,beta,t[n],delta_t)
        E_3 = expectation_fm(I,x,sigma,beta,delta_t,5000,f,IZ)
        Z[n]=E_2/delta_t
        Y[n]=E_1+delta_t*E_3
        I=lagrange(x,Y[n])
        IZ=lagrange(x,Z[n])
    y[N]=Y[0][2]
    z[N]=Z[0][2]

In [32]:
exact_y,exact_z=g(0,0),Z_exact(0,0)

In [33]:
table = PrettyTable()
table.field_names = ["N","y_exact","y","z_exact","z","|y-y_exact|", "|z-z_exact|"]
for i in range(len(N_)):
    table.add_row([N_[i],exact_y,y[i],exact_z,z[i],"{:.8f}".format(abs(y[i]-exact_y)),"{:.8f}".format(abs(z[i]-exact_z))])
print(table)

+---+---------+---------------------+---------+--------------------+-------------+-------------+
| N | y_exact |          y          | z_exact |         z          | |y-y_exact| | |z-z_exact| |
+---+---------+---------------------+---------+--------------------+-------------+-------------+
| 1 |   0.0   |  0.3696406320612634 |   1.0   | 0.2598145988685001 |  0.36964063 |  0.74018540 |
| 2 |   0.0   | 0.35636944084818767 |   1.0   | 0.5410198844725465 |  0.35636944 |  0.45898012 |
| 3 |   0.0   |  0.2979929318608374 |   1.0   | 0.8026502883623208 |  0.29799293 |  0.19734971 |
| 4 |   0.0   | 0.22123685320943262 |   1.0   | 0.9434306472544745 |  0.22123685 |  0.05656935 |
| 5 |   0.0   | 0.16312507452432248 |   1.0   | 1.0345053324955502 |  0.16312507 |  0.03450533 |
+---+---------+---------------------+---------+--------------------+-------------+-------------+


**Reference:** Weidong Zhao, Lifeng Chen, and Shige Peng, New Kind of Accurate Numerical Method for Backward Stochastic Differential Equations, SIAM J. Sci. Comput., 28(4), 1563–1581. (19 pages)

Read More: https://epubs.siam.org/doi/abs/10.1137/05063341X?mobileUi=0

**(4) Consider the decoupled FBSDE:**$$
\left\{\begin{aligned}
d x_{t} &=\sin \left(t+x_{t}\right) d t+\frac{3}{10} \cos \left(t+x_{t}\right) d W_{t} \\
-d y_{t} &=\left(\frac{3}{20} y_{t} z_{t}-\cos \left(t+x_{t}\right)\left(1+y_{t}\right)\right) d t-z_{t} d W_{t}
\end{aligned}\right.
$$

We assume $
x_{0}=1, T=1
$,
with the terminal conditions:$$
y_{T}=\sin \left(T+x_{T}\right)
$$
The analytical solution: $$
\left(y_{t}, z_{t}\right)=\left(\sin \left(t+x_{t}\right), \frac{3}{10} \cos ^{2}\left(t+x_{t}\right)\right)
$$
then we have the below definition:

In [34]:
def g(x,t):
    return np.sin(t+x)

In [35]:
def Z_exact(x,t):
    return (3/10)*(np.cos(t+x)**2)

In [36]:
def f(y,x,t,z):
    return 3*y*z/20-np.cos(t+x)*(1+y)

In [37]:
def sigma(x,t):
    return (3/10)*np.cos(t+x)

In [38]:
def beta(x,t):
    return np.sin(t+x)

In [39]:
def expectation(I,x,sigma,beta,t,delta_t):
    a,w=np.polynomial.hermite.hermgauss(10)
    x = np.expand_dims(x, axis=1)
    a = np.expand_dims(a, axis=0)
    E_1=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a)),w)/np.sqrt(np.pi)
    E_2=np.dot(I(x+beta(x,t)*delta_t+np.dot(sigma(x,t)*np.sqrt(2*delta_t),a))*np.sqrt(2*delta_t)*a,w)/np.sqrt(np.pi)
    return E_1,E_2

In [40]:
def expectation_fm(I,x,sigma,beta,delta_t,NE,f,z,t):
    e3=0
    for i in range(0,NE):
        e3+=f(I(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)),x,t,z(x+beta(x,t)*delta_t+sigma(x,t)*np.sqrt(delta_t)*np.random.normal(0,1)))
    e3/=NE
    return e3

In [41]:
x=np.arange(0.9,1.1,0.05)

In [42]:
x

array([0.9 , 0.95, 1.  , 1.05, 1.1 ])

In [43]:
N_=[1,2,3,4,5,6,7,8,9];T0=0;T=1;
y=np.zeros(len(N_))
z=np.zeros(len(N_))
for N in range(len(N_)):
    t,delta_t=np.linspace(T0,T,num=N_[N]+1,endpoint=True,retstep=True)
    Y=np.zeros((N_[N]+1,len(x)))
    Z=np.zeros((N_[N]+1,len(x)))
    Y[-1]=g(x,T)
    Z[-1]=Z_exact(x,T)
    I=lagrange(x,g(x,T))
    IZ=lagrange(x,Z_exact(x,T))
    for n in range(N_[N]-1,-1,-1): 
        E_1,E_2=expectation(I,x,sigma,beta,t[n],delta_t)
        E_3 = expectation_fm(I,x,sigma,beta,delta_t,500,f,IZ,t[n])
        Z[n]=E_2/delta_t
        Y[n]=E_1+delta_t*E_3
        I=lagrange(x,Y[n])
        IZ=lagrange(x,Z[n])
    y[N]=Y[0][2]
    z[N]=Z[0][2]

In [44]:
exact_y,exact_z=g(1,0),Z_exact(1,0)

In [45]:
table = PrettyTable()
table.field_names = ["N","y_exact","y","z_exact","z","|y-y_exact|", "|z-z_exact|"]
for i in range(len(N_)):
    table.add_row([N_[i],"{:.8f}".format(exact_y),y[i],"{:.8f}".format(exact_z),z[i],"{:.8f}".format(abs(y[i]-exact_y)),"{:.8f}".format(abs(z[i]-exact_z))])
print(table)

+---+------------+----------------------+------------+-----------------------+-------------+-------------+
| N |  y_exact   |          y           |  z_exact   |           z           | |y-y_exact| | |z-z_exact| |
+---+------------+----------------------+------------+-----------------------+-------------+-------------+
| 1 | 0.84147098 | -0.39434256348992663 | 0.08757797 |  -0.1504663677214098  |  1.23581355 |  0.23804434 |
| 2 | 0.84147098 |  0.0745601783036734  | 0.08757797 |  -0.05263065884706262 |  0.76691081 |  0.14020863 |
| 3 | 0.84147098 |  0.2951743626551819  | 0.08757797 | -0.006287597612811497 |  0.54629662 |  0.09386557 |
| 4 | 0.84147098 |  0.4203086809923051  | 0.08757797 |  0.018218393336173734 |  0.42116230 |  0.06935958 |
| 5 | 0.84147098 |  0.4978103985163044  | 0.08757797 |  0.03210878694347611  |  0.34366059 |  0.05546919 |
| 6 | 0.84147098 |  0.5509177832302845  | 0.08757797 |  0.04100438153925775  |  0.29055320 |  0.04657359 |
| 7 | 0.84147098 |  0.590725164117488

**Reference:** Tao Tang, Weidong Zhao and Tao Zhou, Deferred Correction Methods for Forward Backward Stochastic Differential Equations, Numerical Mathematics: Theory, Methods and Applications , Volume 10 , Issue 2 , May 2017 , pp. 222 - 242
DOI: https://doi.org/10.4208/nmtma.2017.s02